In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
#https://data.mendeley.com/datasets/c3xvj5snvv/1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_csv("/content/drive/My Drive/Traffic_data.csv")
df

,Text,Label
0,UPDATE: Rt. in York County has reopened near C...,2
1,#Fargo #ND #USA - Sr. Clinical Research Associ...,0
2,Just two of the biggest fans riding the ride i...,0
3,Wreck on Southbound just north of Atlanta High...,1
4,Justin bieber's new album is going to be amazi...,0
...,...,...
40937,Start beach-bonfire-BBQ season off right with ...,0
40938,Snow quickly covering roads in the southern co...,2
40939,Updated: Crash in Duval on highway east at Exi...,1
40940,Join the Truth Team and help set the record st...,0


In [ ]:
df['label'] = np.where(df['Label'] == 1, 'Traffic Incident or Accident Related', 'Not Traffic Incident nor Accident Related')
df

,Text,Label,label
0,UPDATE: Rt. in York County has reopened near C...,2,Not Traffic Incident nor Accident Related
1,#Fargo #ND #USA - Sr. Clinical Research Associ...,0,Not Traffic Incident nor Accident Related
2,Just two of the biggest fans riding the ride i...,0,Not Traffic Incident nor Accident Related
3,Wreck on Southbound just north of Atlanta High...,1,Traffic Incident or Accident Related
4,Justin bieber's new album is going to be amazi...,0,Not Traffic Incident nor Accident Related
...,...,...,...
40937,Start beach-bonfire-BBQ season off right with ...,0,Not Traffic Incident nor Accident Related
40938,Snow quickly covering roads in the southern co...,2,Not Traffic Incident nor Accident Related
40939,Updated: Crash in Duval on highway east at Exi...,1,Traffic Incident or Accident Related
40940,Join the Truth Team and help set the record st...,0,Not Traffic Incident nor Accident Related


In [ ]:
df=df[0:1000]
df

,Text,Label,label
0,UPDATE: Rt. in York County has reopened near C...,2,Not Traffic Incident nor Accident Related
1,#Fargo #ND #USA - Sr. Clinical Research Associ...,0,Not Traffic Incident nor Accident Related
2,Just two of the biggest fans riding the ride i...,0,Not Traffic Incident nor Accident Related
3,Wreck on Southbound just north of Atlanta High...,1,Traffic Incident or Accident Related
4,Justin bieber's new album is going to be amazi...,0,Not Traffic Incident nor Accident Related
...,...,...,...
995,Accident in the Lowry Hill Tunnel affecting bo...,1,Traffic Incident or Accident Related
996,"Gilbert Street &amp; University Blvd., East, t...",1,Traffic Incident or Accident Related
997,Gatekeeper of #DC Society Sally Quinn Comes Ou...,0,Not Traffic Incident nor Accident Related
998,Cleared: Incident on #ELine Manhattan bound at...,2,Not Traffic Incident nor Accident Related


In [ ]:
pip install transformers

In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00


In [ ]:
import torch

if torch.cuda.is_available():
    print("GPU is available:", torch.cuda.get_device_name(0))
else:
    print("GPU is not available.")


GPU is available: Tesla T4


In [ ]:
#1.model_name = "bert-base-uncased"
#2.model_name = "bert-large-uncased"

#3.model_name = "roberta-large"

#4.model_name = "facebook/bart-large"
#5.model_name="facebook/bart-large-mnli"
#6.model_name ="valhalla/distilbart-mnli-12-1"
#7.model_name="cross-encoder/nli-distilroberta-base"

#8.model_name = "gpt2"

In [ ]:
#1. "bert-base-uncased"
from transformers import pipeline
from datasets import Dataset
device = 0 if torch.cuda.is_available() else -1  # if GPU is available, use it, otherwise use CPU
# Set up the zero-shot classification pipeline with the BERT model and GPU
model_name = "bert-base-uncased"
classifier = pipeline("zero-shot-classification", model=model_name, device=device)

# Convert the DataFrame to a Hugging Face Dataset
df_dataset = Dataset.from_pandas(df)

# Define the lables
labels = ['Traffic Incident or Accident Related', 'Not Traffic Incident nor Accident Related']

hypothesis_template = "This text is {}."
#hypothesis_template=hypothesis_template
# Function to apply the classifier to each batch
def apply_classifier(batch):
    results = classifier(batch['Text'], labels,hypothesis_template=hypothesis_template)
    return {'Predicted_label': [result["labels"][0] for result in results]}

# Apply the function using map in batches
df_dataset = df_dataset.map(apply_classifier, batched=True, batch_size=8)

# Convert the dataset back to a pandas DataFrame
df = df_dataset.to_pandas()

# Now compare the predicted genres with the actual genres to calculate the accuracy
correct_predictions = sum(df['Predicted_label'].str.lower() == df['label'].str.lower())
accuracy = correct_predictions / len(df)
print(f"Accuracy: {accuracy * 100:.2f}%")
#57.39% 42.2

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Accuracy: 68.70%


In [ ]:
#2. "bert-large-uncased"
from transformers import pipeline
from datasets import Dataset
device = 0 if torch.cuda.is_available() else -1  # if GPU is available, use it, otherwise use CPU
# Set up the zero-shot classification pipeline with the BERT model and GPU
model_name = "bert-large-uncased"
classifier = pipeline("zero-shot-classification", model=model_name, device=device)

# Convert the DataFrame to a Hugging Face Dataset
df_dataset = Dataset.from_pandas(df)

# Define the lables
labels = ['Traffic Incident or Accident Related', 'Not Traffic Incident nor Accident Related']

hypothesis_template = "This text is {}."
#hypothesis_template=hypothesis_template
# Function to apply the classifier to each batch
def apply_classifier(batch):
    results = classifier(batch['Text'], labels,hypothesis_template=hypothesis_template)
    return {'Predicted_label': [result["labels"][0] for result in results]}

# Apply the function using map in batches
df_dataset = df_dataset.map(apply_classifier, batched=True, batch_size=8)

# Convert the dataset back to a pandas DataFrame
df = df_dataset.to_pandas()

# Now compare the predicted genres with the actual genres to calculate the accuracy
correct_predictions = sum(df['Predicted_label'].str.lower() == df['label'].str.lower())
accuracy = correct_predictions / len(df)
print(f"Accuracy: {accuracy * 100:.2f}%")
#58.54%

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Accuracy: 53.70%


In [ ]:
#3. model_name = "roberta-large"
from transformers import pipeline
from datasets import Dataset
device = 0 if torch.cuda.is_available() else -1  # if GPU is available, use it, otherwise use CPU
# Set up the zero-shot classification pipeline with the BERT model and GPU
model_name = "roberta-large"
classifier = pipeline("zero-shot-classification", model=model_name, device=device)

# Convert the DataFrame to a Hugging Face Dataset
df_dataset = Dataset.from_pandas(df)

# Define the lables
labels = ['Traffic Incident or Accident Related', 'Not Traffic Incident nor Accident Related']

hypothesis_template = "This text is {}."
#hypothesis_template=hypothesis_template
# Function to apply the classifier to each batch
def apply_classifier(batch):
    results = classifier(batch['Text'], labels,hypothesis_template=hypothesis_template)
    return {'Predicted_label': [result["labels"][0] for result in results]}

# Apply the function using map in batches
df_dataset = df_dataset.map(apply_classifier, batched=True, batch_size=8)

# Convert the dataset back to a pandas DataFrame
df = df_dataset.to_pandas()

# Now compare the predicted genres with the actual genres to calculate the accuracy
correct_predictions = sum(df['Predicted_label'].str.lower() == df['label'].str.lower())
accuracy = correct_predictions / len(df)
print(f"Accuracy: {accuracy * 100:.2f}%")
#57.39% 42.2

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Accuracy: 64.50%


In [ ]:
#4.model_name = "facebook/bart-large"
from transformers import pipeline
from datasets import Dataset
device = 0 if torch.cuda.is_available() else -1  # if GPU is available, use it, otherwise use CPU
# Set up the zero-shot classification pipeline with the BERT model and GPU
model_name = "facebook/bart-large"
classifier = pipeline("zero-shot-classification", model=model_name, device=device)

# Convert the DataFrame to a Hugging Face Dataset
df_dataset = Dataset.from_pandas(df)

# Define the lables
labels = ['Traffic Incident or Accident Related', 'Not Traffic Incident nor Accident Related']

hypothesis_template = "This text is {}."
#hypothesis_template=hypothesis_template
# Function to apply the classifier to each batch
def apply_classifier(batch):
    results = classifier(batch['Text'], labels,hypothesis_template=hypothesis_template)
    return {'Predicted_label': [result["labels"][0] for result in results]}

# Apply the function using map in batches
df_dataset = df_dataset.map(apply_classifier, batched=True, batch_size=8)

# Convert the dataset back to a pandas DataFrame
df = df_dataset.to_pandas()

# Now compare the predicted genres with the actual genres to calculate the accuracy
correct_predictions = sum(df['Predicted_label'].str.lower() == df['label'].str.lower())
accuracy = correct_predictions / len(df)
print(f"Accuracy: {accuracy * 100:.2f}%")
#57.39% 42.2

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-large and are newly initialized: ['classification_head.out_proj.bias', 'classification_head.dense.bias', 'classification_head.out_proj.weight', 'classification_head.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Accuracy: 50.90%


In [ ]:
#5.model_name="facebook/bart-large-mnli"
from transformers import pipeline
from datasets import Dataset
device = 0 if torch.cuda.is_available() else -1  # if GPU is available, use it, otherwise use CPU
# Set up the zero-shot classification pipeline with the BERT model and GPU
model_name="facebook/bart-large-mnli"
classifier = pipeline("zero-shot-classification", model=model_name, device=device)

# Convert the DataFrame to a Hugging Face Dataset
df_dataset = Dataset.from_pandas(df)

# Define the lables
labels = ['Traffic Incident or Accident Related', 'Not Traffic Incident nor Accident Related']

hypothesis_template = "This text is {}."
#hypothesis_template=hypothesis_template
# Function to apply the classifier to each batch
def apply_classifier(batch):
    results = classifier(batch['Text'], labels,hypothesis_template=hypothesis_template)
    return {'Predicted_label': [result["labels"][0] for result in results]}

# Apply the function using map in batches
df_dataset = df_dataset.map(apply_classifier, batched=True, batch_size=8)

# Convert the dataset back to a pandas DataFrame
df = df_dataset.to_pandas()

# Now compare the predicted genres with the actual genres to calculate the accuracy
correct_predictions = sum(df['Predicted_label'].str.lower() == df['label'].str.lower())
accuracy = correct_predictions / len(df)
print(f"Accuracy: {accuracy * 100:.2f}%")
#57.39% 42.2

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Accuracy: 36.00%


In [ ]:
#6.model_name ="valhalla/distilbart-mnli-12-1"
from transformers import pipeline
from datasets import Dataset
device = 0 if torch.cuda.is_available() else -1  # if GPU is available, use it, otherwise use CPU
# Set up the zero-shot classification pipeline with the BERT model and GPU
model_name ="valhalla/distilbart-mnli-12-1"
classifier = pipeline("zero-shot-classification", model=model_name, device=device)

# Convert the DataFrame to a Hugging Face Dataset
df_dataset = Dataset.from_pandas(df)

# Define the lables
labels = ['Traffic Incident or Accident Related', 'Not Traffic Incident nor Accident Related']

hypothesis_template = "This text is {}."
#hypothesis_template=hypothesis_template
# Function to apply the classifier to each batch
def apply_classifier(batch):
    results = classifier(batch['Text'], labels,hypothesis_template=hypothesis_template)
    return {'Predicted_label': [result["labels"][0] for result in results]}

# Apply the function using map in batches
df_dataset = df_dataset.map(apply_classifier, batched=True, batch_size=8)

# Convert the dataset back to a pandas DataFrame
df = df_dataset.to_pandas()

# Now compare the predicted genres with the actual genres to calculate the accuracy
correct_predictions = sum(df['Predicted_label'].str.lower() == df['label'].str.lower())
accuracy = correct_predictions / len(df)
print(f"Accuracy: {accuracy * 100:.2f}%")
#57.39% 42.2

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/890M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Accuracy: 74.00%


In [ ]:
#7.model_name="cross-encoder/nli-distilroberta-base"
from transformers import pipeline
from datasets import Dataset
device = 0 if torch.cuda.is_available() else -1  # if GPU is available, use it, otherwise use CPU
# Set up the zero-shot classification pipeline with the BERT model and GPU
model_name="cross-encoder/nli-distilroberta-base"
classifier = pipeline("zero-shot-classification", model=model_name, device=device)

# Convert the DataFrame to a Hugging Face Dataset
df_dataset = Dataset.from_pandas(df)

# Define the lables
labels = ['Traffic Incident or Accident Related', 'Not Traffic Incident nor Accident Related']

hypothesis_template = "This text is {}."
#hypothesis_template=hypothesis_template
# Function to apply the classifier to each batch
def apply_classifier(batch):
    results = classifier(batch['Text'], labels,hypothesis_template=hypothesis_template)
    return {'Predicted_label': [result["labels"][0] for result in results]}

# Apply the function using map in batches
df_dataset = df_dataset.map(apply_classifier, batched=True, batch_size=8)

# Convert the dataset back to a pandas DataFrame
df = df_dataset.to_pandas()

# Now compare the predicted genres with the actual genres to calculate the accuracy
correct_predictions = sum(df['Predicted_label'].str.lower() == df['label'].str.lower())
accuracy = correct_predictions / len(df)
print(f"Accuracy: {accuracy * 100:.2f}%")
#57.39% 42.2

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Accuracy: 47.70%


In [ ]:
#8.model_name = "gpt2"
from transformers import pipeline
from datasets import Dataset
device = 0 if torch.cuda.is_available() else -1  # if GPU is available, use it, otherwise use CPU
# Set up the zero-shot classification pipeline with the BERT model and GPU
model_name = "gpt2"
classifier = pipeline("zero-shot-classification", model=model_name, device=device)

# Convert the DataFrame to a Hugging Face Dataset
df_dataset = Dataset.from_pandas(df)

# Define the lables
labels = ['Traffic Incident or Accident Related', 'Not Traffic Incident nor Accident Related']

hypothesis_template = "This text is {}."
#hypothesis_template=hypothesis_template
# Function to apply the classifier to each batch
def apply_classifier(batch):
    results = classifier(batch['Text'], labels,hypothesis_template=hypothesis_template)
    return {'Predicted_label': [result["labels"][0] for result in results]}

# Apply the function using map in batches
df_dataset = df_dataset.map(apply_classifier, batched=True, batch_size=8)

# Convert the dataset back to a pandas DataFrame
df = df_dataset.to_pandas()

# Now compare the predicted genres with the actual genres to calculate the accuracy
correct_predictions = sum(df['Predicted_label'].str.lower() == df['label'].str.lower())
accuracy = correct_predictions / len(df)
print(f"Accuracy: {accuracy * 100:.2f}%")
#57.39% 42.2

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizer was not supporting padding necessary for zero-shot, attempting to use  `pad_token=eos_token`
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Accuracy: 34.30%
